In [1]:
#install packages
import pandas as pd
import os

In [74]:
data_path = "./data/Digital_Music_5.json"
table = pd.read_json(data_path, 'records', lines = True).reset_index(drop = True, inplace = False)
table.tail()

C:\Users\lky\AppData\Local\Temp\ipykernel_33140\2885759717.py:2: FutureWarning: Starting with pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only.
  table = pd.read_json(data_path, 'records', lines = True).reset_index(drop = True, inplace = False)


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
64701,A1PQ1PESSO8CMO,B00KILDVEI,Ginger Christmas,"[0, 0]",I like the reggae sound a lot in this song. I ...,4,Cool song,1403568000,"06 24, 2014"
64702,A120RH58WVY4W6,B00KILDVEI,"Kelly Dunwell ""avid reader""","[0, 0]",I first heard this on Sirius and had to have i...,5,Great Song,1404864000,"07 9, 2014"
64703,A19VJ2IQLO50G0,B00KILDVEI,melinda,"[0, 1]","I absolutely love this song, it downloaded fin...",5,Five Stars,1405209600,"07 13, 2014"
64704,AUDSM2CTLLW1Q,B00KILDVEI,Patrick L. Randall,"[0, 0]","Reggae, island beats aren't really my cup of t...",3,Well-crafted song,1404864000,"07 9, 2014"
64705,A1GN8UJIZLCA59,B00KILDVEI,P Magnum,"[1, 2]",Magic! is a Canadian band that incorporates re...,1,Souless Reggae,1405641600,"07 18, 2014"


In [75]:
import time

tic = time.time()
for i in range(10000):
    a = 1

toc = time.time()
print(toc - tic)

0.0014045238494873047


In [76]:
# transform unixReviewTime to datetime
from datetime import datetime

condition = lambda row: datetime.fromtimestamp(row).strftime("%m-%d-%Y")
table["unixReviewTime"] = table["unixReviewTime"].apply(condition)
table.drop(labels="reviewTime", axis=1, inplace=True)

display(table.head())

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime
0,A3EBHHCZO6V2A4,5555991584,"Amaranth ""music fan""","[3, 3]","It's hard to believe ""Memory of Trees"" came ou...",5,Enya's last great album,09-12-2006
1,AZPWAXJG9OJXV,5555991584,bethtexas,"[0, 0]","A clasically-styled and introverted album, Mem...",5,Enya at her most elegant,06-03-2001
2,A38IRL0X2T4DPF,5555991584,bob turnley,"[2, 2]",I never thought Enya would reach the sublime h...,5,The best so far,07-14-2003
3,A22IK3I6U76GX0,5555991584,Calle,"[1, 1]",This is the third review of an irish album I w...,5,Ireland produces good music.,05-03-2000
4,A1AISPOIIHTHXX,5555991584,"Cloud ""...""","[1, 1]","Enya, despite being a successful recording art...",4,4.5; music to dream to,01-17-2008


In [47]:
print(table.overall.unique())

[5 4 3 1 2]


In [77]:
#remove all the punctation since it doesn't work in the way we'll perform NLP
pattern = '[^\w\s]'
table['preprocessed'] = table['reviewText'].str.replace(pat=pattern, repl="", regex=True)
print(table["preprocessed"].iloc[0])

Its hard to believe Memory of Trees came out 11 years agoit has held up well over the passage of timeIts Enyas last great album before the New Agepop of Amarantine and Day without rain Back in 1995Enya still had her creative sparkher own voiceI agree with the reviewer who said that this is her saddest albumit is melancholybittersweetfrom the opening title songMemory of Trees is elegaicmajesticPax Deorum sounds like it is from a Requiem Massit is a dark threnodyUnlike the reviewer who said that this has a disconcerting blend of spiritualitysensualityI dont find it disconcerting at allAnywhere is is a hopeful songlooking to possibilitiesHope has a place is about lovebut it is up to the listener to decide if it is romanticplatonicetcIve always had a soft spot for this songOn my way home is a triumphant ending about returnThis is truly a masterpiece of New Age musica must for any Enya fan


In [78]:
#convert to lower case
table['preprocessed'] = table['preprocessed'].str.lower()
print(table["preprocessed"].iloc[0])

its hard to believe memory of trees came out 11 years agoit has held up well over the passage of timeits enyas last great album before the new agepop of amarantine and day without rain back in 1995enya still had her creative sparkher own voicei agree with the reviewer who said that this is her saddest albumit is melancholybittersweetfrom the opening title songmemory of trees is elegaicmajesticpax deorum sounds like it is from a requiem massit is a dark threnodyunlike the reviewer who said that this has a disconcerting blend of spiritualitysensualityi dont find it disconcerting at allanywhere is is a hopeful songlooking to possibilitieshope has a place is about lovebut it is up to the listener to decide if it is romanticplatonicetcive always had a soft spot for this songon my way home is a triumphant ending about returnthis is truly a masterpiece of new age musica must for any enya fan


In [79]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.cluster.util import cosine_distance
from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import wordnet

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

import re

In [80]:
# lemmatize
def lemmatize_word(tagged_token):
    root = []
    lemmatizer = WordNetLemmatizer()
    for token in tagged_token:
        tag = token[1][0].lower()
        tag = tag if tag in ['a', 'r', 'n', 'v'] else None
        word = token[0]
        if not tag:
            root.append(word)
        else:
            root.append(lemmatizer.lemmatize(word, tag))
    return root

def lemmatize_doc(row):
    lemmatized_list = []
    tokenized_sent = sent_tokenize(row)
    for sentence in tokenized_sent:
        no_punctuation = re.sub(r'[^\w\s]', ' ', sentence)
        tokenized_word = word_tokenize(no_punctuation)
        tagged_token = pos_tag(tokenized_word)
        lemmatized = lemmatize_word(tagged_token)
        lemmatized_list.extend(lemmatized)
    return " ".join(lemmatized_list)

table['lemmatized'] = table['preprocessed'].head(1000).apply(lambda row: lemmatize_doc(row))

print(table['lemmatized'].iloc[0])

12.629493474960327
its hard to believe memory of tree come out 11 year agoit have hold up well over the passage of timeits enyas last great album before the new agepop of amarantine and day without rain back in 1995enya still have her creative sparkher own voicei agree with the reviewer who say that this be her saddest albumit be melancholybittersweetfrom the opening title songmemory of tree be elegaicmajesticpax deorum sound like it be from a requiem massit be a dark threnodyunlike the reviewer who say that this have a disconcert blend of spiritualitysensualityi dont find it disconcert at allanywhere be be a hopeful songlooking to possibilitieshope have a place be about lovebut it be up to the listener to decide if it be romanticplatonicetcive always have a soft spot for this songon my way home be a triumphant end about returnthis be truly a masterpiece of new age musica must for any enya fan


In [ ]:
table['lemmatized'].apply(lambda x: print(x))